In [ ]:
# cleaning...%load_ext autoreload
# cleaning...%autoreload 2

In [ ]:
import pandas as pd

import kraft

In [ ]:
for size in (
    (1, 8),
    (2, 8),
    (4, 8),
    (8, 8),
    (16, 8),
    (32, 8),
    (64, 8),
    (128, 8),
    (256, 8),
    (8, 2),
    (8, 4),
    (8, 16),
    (8, 32),
    (8, 64),
    (8, 128),
    (8, 256),
):

    target = kraft.simulate_series_or_dataframe("Column", "Target", size[1], "uniform")

    data = kraft.simulate_series_or_dataframe("Index", "Column", size, "uniform")

    kraft.make_match_panel(target, data, n_extreme=None)

In [ ]:
size = (1, 10)

target = kraft.simulate_series_or_dataframe("Column", "Target", size[1], "uniform")

data = kraft.simulate_series_or_dataframe("Index", "Column", size, "uniform")

for n_permutation, n_sampling in ((0, 0), (0, 10), (10, 0), (10, 10)):

    kraft.make_match_panel(
        target, data, n_permutation=n_permutation, n_sampling=n_sampling
    )

In [ ]:
size = (10, 20)

target = kraft.simulate_series_or_dataframe("Column", "Target", size[1], "uniform")

data = kraft.simulate_series_or_dataframe(
    "Index", "Column", size, "uniform", break_dataframe=2
)

kraft.make_match_panel(target, data)

In [ ]:
data = kraft.simulate_series_or_dataframe(
    "Index", "Column", (20, 10), "uniform", break_dataframe=2
)

for target_vector in (
    ((1,) * 1 + (2,) * 2 + (0,) * 7),
    ((2,) * 2 + (3,) * 3 + (0,) * 5),
):

    target = pd.Series(target_vector, index=data.columns)

    for target_ascending in (None, True, False):

        kraft.make_match_panel(
            target,
            data,
            target_ascending=target_ascending,
            target_data_type="categorical",
        )

In [ ]:
target = pd.read_csv("target.tsv", sep="\t", index_col=0, squeeze=True)

data = pd.read_csv("data.tsv", sep="\t", index_col=0)

kraft.make_match_panel(
    target, data, n_sampling=0, n_permutation=0, target_data_type="binary"
)

kraft.make_match_panel(
    data.iloc[0], data.iloc[:1], n_sampling=0, n_permutation=0, plot_std=3
)

kraft.make_match_panel(
    data.iloc[0], data.iloc[:2], n_sampling=0, n_permutation=0, plot_std=3
)

In [ ]:
target = kraft.simulate_series_or_dataframe("Column", "Target", 10, "uniform")

data_continuous = kraft.simulate_series_or_dataframe(
    "Index", "Column", (2, 10), "uniform"
)

data_categorical = kraft.simulate_series_or_dataframe("Index", "Column", (4, 10), 3)

data_binary = kraft.simulate_series_or_dataframe("Index", "Column", (8, 10), 2)

data_continuous.drop(data_continuous.columns[-1], axis=1, inplace=True)

data_categorical.drop(data_categorical.columns[-2], axis=1, inplace=True)

data_binary.drop(data_binary.columns[-3], axis=1, inplace=True)

data_dicts = {
    "Continuous Data": {"dataframe": data_continuous, "data_type": "continuous"},
    "Categorical Data": {"dataframe": data_categorical, "data_type": "categorical"},
    "Binary Data": {"dataframe": data_binary, "data_type": "binary"},
}

for plot_only_shared_by_target_and_all_data in (False, True):

    kraft.make_match_panel_summary(
        target,
        data_dicts,
        {
            data_name: kraft.make_match_panel(
                target, data_dict["dataframe"], plot=False
            )
            for data_name, data_dict in data_dicts.items()
        },
        plot_only_shared_by_target_and_all_data=plot_only_shared_by_target_and_all_data,
    )